In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & \alpha + i \beta \\ \alpha - i \beta & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$



In [6]:
# Define polynomial variables
@polyvar w α β r ϕ e p

#Hˢʸᵐᵇ = [ w   0
#          0   0. ]

Hˢʸᵐᵇ = [ w           α + im* β
          α + im* β   0.        ]

@polyvar a[1:4]
@polyvar b[1:4]

#"just-zero-down_ "

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2] + im*b[2] 
#          0                -a[1]-im*b[1]  ]

J₁ˢʸᵐᵇ = [ a[1]+im*b[1]      r
          a[3] + im*b[3]   -a[1]-im*b[1] ]
          

#J₁ˢʸᵐᵇ = [ 0   r
#           0   0. ]

#J₂ˢʸᵐᵇ = ϕ * [ 1    0
#               0   -1. ]

#J₃ˢʸᵐᵇ = [ 0   0
#           e   0. ]

#J₄ˢʸᵐᵇ = p * [ 0    1
#               1    0. ]

#J₅ˢʸᵐᵇ = p * [ 0   -im
#               im   0  ]

#J₆ˢʸᵐᵇ = p * [ 1    0
#               0   -1. ]


function g_objective_full(γᵢ)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [J₁ˢʸᵐᵇ])
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective_full (generic function with 1 method)

In [3]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

local_rand_min (generic function with 1 method)

In [4]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;


../DATA/


In [7]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "POP_LME_1_full_trn4_tst20_"*date_and_time_string * ".h5"

for γᵢ in γ

    println("γ =  "*γᵢ)

    poly = g_objective_full(γᵢ)
    
    sol = local_rand_min(poly)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    relaxation = subs(r^2, sol)
    #de_phasing = subs(ϕ^2, sol)
    #exitation = subs(e^2, sol) 
    #de_polarisation = subs(p^2, sol)
    #omega = subs(w, sol)

    J₁ˢⁱᵈ = subs(J₁ˢʸᵐᵇ, sol)
    #J₂ˢⁱᵈ = subs(J₂ˢʸᵐᵇ, sol)
    #J₃ˢⁱᵈ = subs(J₃ˢʸᵐᵇ, sol) 

    #J₄ˢⁱᵈ = subs(J₄ˢʸᵐᵇ, sol)
    #J₅ˢⁱᵈ = subs(J₅ˢʸᵐᵇ, sol)
    #J₆ˢⁱᵈ = subs(J₆ˢʸᵐᵇ, sol)


    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group   

        γ_group["omega"] = convert(Float64, omega)
        γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        #γ_group["gamma_de_phasing"] = convert(Float64,  de_phasing)
        #γ_group["gamma_exitation"] = convert(Float64,  exitation)
        #γ_group["gamma_de_polarisation"] = convert(Float64, de_polarisation)
        
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["J1"] = convert.(ComplexF64, J₁ˢⁱᵈ)
        #γ_group["J2"] = convert.(ComplexF64, J₂ˢⁱᵈ)
        #γ_group["J3"] = convert.(ComplexF64, J₃ˢⁱᵈ)
        #γ_group["J4"] = convert.(ComplexF64, J₄ˢⁱᵈ)
        #γ_group["J5"] = convert.(ComplexF64, J₅ˢⁱᵈ)
        #γ_group["J6"] = convert.(ComplexF64, J₆ˢⁱᵈ)
        
    end

    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), [DenseOperator(basis,J₁ˢⁱᵈ)])
        
        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            
        end
                    
        println(minimum(Fᴸᴹᴱₑₓ))
    
    end

    println()
end

println(tests_data_file_name)

γ =  0.079477


State_D1 0.9892250543702875


State_D2 

0.9892801622323361
State_D3 0.9984523769683096


State_D4 

...


State_D20 0.973402592895207

γ =  251.33
State_D1 

0.9971869998895051
State_D2 0.9975225389997803
State_D3 0.9969753273237055
State_D4 0.9974387432713698
State_D5 0.9966836055451347
State_D6 0.994652576523633
State_D7 

0.9970766970194005
State_D8 0.9938792190818815
State_D9 0.996876251143655
State_D10 0.9966623089174331
State_D11 0.9953415280936826
State_D12 0.9969107442959777
State_D13 0.9953664579468267
State_D14 0.9973263346559695


State_D15 0.9945690774749687
State_D16 0.9966099996650376
State_D17 0.9921494750522544
State_D18 0.9980707868548735
State_D19 0.9964629622086333
State_D20 0.9966311375996522



POP_LME_1_full_trn4_tst20_2024-Jan-17_at_11-29.h5
